In [67]:
from bs4 import BeautifulSoup
from requests import get
from pandas import*
from nltk import sent_tokenize
import nltk
from nltk import word_tokenize
import pandas as pd

In [68]:
class RequestBeautifulSoup:
    def __init__(self,url):
        self.url = url
        self.intervalos = None
        self.perguntas = None
        self.respostas = None
    
    # Request do soup da biblioteza BeautifulSoup
    def requestSoup(self):
        response = get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    # Intervalos de onde estão as respostas
    def getIntervalos(self,listaPrincipal):
        intervalos = []
        for i in range(len(self.perguntas[1])):
            if(i != len(self.perguntas[1])-1):
                intervalos.append((self.perguntas[1][i]+1,self.perguntas[1][i+1]))
            else:
                intervalos.append((self.perguntas[1][i]+1,len(listaPrincipal)))
        self.intervalos = intervalos

    # Perguntas e suas posições
    def getPerguntas(self,soup, chaves = None):
        perguntas = []
        posi = []
        for i in range(len(soup)):
            if(("?" in soup[i].text) and ('https' not in soup[i].text)):
                aux = soup[i].text.replace('\r','').replace('\n','').replace('\t','').replace('\xa0','')
                if(("?" in sent_tokenize(aux)[0]) and (":" not in sent_tokenize(aux)[0])):
                    if(chaves is not None):
                        texto = soup[i].text
                        for j in chaves:
                            texto = texto.replace(j,"")
                        perguntas.append(texto)
                        posi.append(i)
                    else:
                        perguntas.append(soup[i].text)
                        posi.append(i)

        self.perguntas = [perguntas,posi]
    
    # Respostas
    def getRespostas(self,soup,chaves = None,subs = ""):
        respostas = []
        for i in self.intervalos:
            text = ""
            for j in range(i[0],i[1]):
                text += soup[j].text
            if(chaves is not None):
                for j in chaves:
                    text = text.replace(j,subs)
                respostas.append(text)
            else:
                respostas.append(text)
        self.respostas = respostas
        
    def getComparacao(self,first,second):
        if(len(first) == len(second)):
            return True
        else:
            return False
        
    def criarDataframePDF(self,perguntas,respostas,salve_name = None):
        if(salve_name is not None):    
            pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./' + salve_name + '.csv', index= False)
        else:
            return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})
    

In [69]:
who = 'https://www.who.int/emergencies/diseases/novel-coronavirus-2019/question-and-answers-hub'
html_soup = get(who)
soup = BeautifulSoup(html_soup.text,'html.parser')

div_soup = soup.find('div',class_ = 'sf-list-vertical sf-list-vertical__nodate')
a_soup = div_soup.find_all('a')

links = []
for href in a_soup:
    links.append("https://www.who.int{}".format(href['href']))

In [70]:
for link,i in zip(links,range(len(links))):
    objeto_soup = RequestBeautifulSoup(link)
    soup = objeto_soup.requestSoup()

    div_soup = soup.find('div',id = 'sf-accordion')

    lista_soup = []
    for conjunto in div_soup.find_all(['a','p','ul','li']):
        lista_soup.append(conjunto)

    objeto_soup.getPerguntas(lista_soup,['\r','\n'])
    perguntas = objeto_soup.perguntas

    novo = []
    for pergunta in perguntas[0]:
        novo.append(' '.join(word_tokenize(pergunta,language= 'english')))

    objeto_soup.perguntas[0] = novo

    objeto_soup.getIntervalos(lista_soup)

    objeto_soup.getRespostas(lista_soup,['\r','\n','\xa0'])

    objeto_soup.criarDataframePDF(objeto_soup.perguntas[0],objeto_soup.respostas,salve_name = str(i))

In [71]:
perguntas = []
respostas = []
for i in range(len(links)):
    df = read_csv('./' + str(i) + '.csv')
    for i in df['Perguntas'].values:
        perguntas.append(i)
    for i in df['Respostas'].values:
        respostas.append(i)

In [72]:
df = DataFrame({'questions' : perguntas, 'answers' : respostas})

In [73]:
df.to_csv("./who.csv",index = False)

In [74]:
for i in range(len(links)):
    df_aux = read_csv('./' + str(i) + '.csv')
    if(df_aux['Respostas'].isna().sum() >= 1):
        print(i)

In [77]:
df

,questions,answers
0,What is COVID-19 ?,COVID-19 is the disease caused by a new corona...
1,What are the symptoms of COVID-19 ?,The most common symptoms of COVID-19 areFeverD...
2,What happens to people who get COVID-19 ?,"Among those who develop symptoms, most (about ..."
3,Who is most at risk of severe illness from COV...,"People aged 60 years and over, and those with ..."
4,Are there long-term effects of COVID-19 ?,"Some people who have had COVID-19, whether the..."
...,...,...
359,Why should I care about violence against women...,Violence against women is a grave violation of...
360,What can staff do to stay safe while working a...,Everyone should follow basic precautions: Wash...
361,Should staff wear a mask while working ?,In areas where the virus that causes COVID-19 ...
362,What should staff do if a guest becomes sick a...,If a guest at the accommodation develops sympt...
